In [ ]:
import os
import numpy as np
import tensorflow as tf

from alexnet import alexnet
from DataGenerator import DataGenerator
from datetime import datetime
from tensorflow.contrib.data import Iterator
from tensorflow.contrib.data.Dataset import Dataset

# data file path in computer 
train_file = " "
valid_file = " "

# parameters for model
learning_rate = 0.01
num_epochs = 100
batch_size = 128
dropout_rate = 0.5
num_classes = 1000
train_layers = ["fc6", "fc7", "fc8"]  # not use the fine-tuned wieghts 

## setting the checkpoint
display_step = 10
# tensorboard and checkpoint path in computer
filewriter_path = " "
checkpoint_path = " "

# check if the tensorboard and checkpoint exist and if not, create the folder
if not os.path.isdir(checkpoint_path):
    os.mkdir(checkpoint_path)
if not os.path.isdir(filewriter_path):
    os.mkdir(filewriter_path)

# methods for data 
train = DataGenerator(train_file, batch_size = batch_size, 
                           num_classes = num_classes)
valid = DataGenerator(valid_file, batch_size = batch_size, 
                          num_classes = num_classes, shuffle = false)

# get the data from train and valid instance
train_data = train.data
valid_data = valid.data

## Iterator with reinitializabel method 
# get the batch data for model
Iter = Iterator.from_structure(train_data.output_type, train_data.output_shape)
next_batch = Iter.get_next()
# Ops for initializing the two iterators
train_init_op = iterator.make_initializer(train_data)
valid_init_op = iterator.make_initializer(valid_data)

# placeholders for alexnet model
x = tf.placeholder(tf.float32, shape = [batch_size, 227, 227, 3])
y = tf.placeholder(tf.float32, shape = [batch_size, num_classes])
keep_pro = tf.placeholder(tf.float32)

# initialize the alexnet model
model = alexnet(x, num_classes, keep_pro, train_layers)

# output of the last fully cinnected layer
out = model.out

# lists for trianable layer (fully connected layers)
var_list = [v for v in tf.trainable_variables() if v.name.split("/")[0] in train_layers]

# Op for loss 
with tf.name_scope("Loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out, labels = y))
    
# build the optimizer for model
with tf.name_scope("train"):
    optimizer = tf.trian.GradientDescentOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss, var_list)    
    gradients = [(MyCapper(gv[0]), gv[1]) for gv in gradients]
    train_op = optimizer.apply_gradients(grads_and_vars = gradients)
    
# Evaluation method: evaluating the model
with tf.name_scope("Accuracy"):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

# summary setting
# gradients
for gradient, var in gradients:
    tf.summary.histogram(var.name + "/gradient:", gradient)
for var in var_list:
    tf.summary.histogram(var.name, var)
# loss
tf.summary.scalar("Loss", loss)
# accuracy
tf.summary.scalar("accuracy", accuracy)
# merge all summaries together
merged_summary = tf.summary.merge_all()

# initialize a filewriter and saver
writer = tf.summary.FileWriter(filewriter_path)
saver = tf.summary.Saver()

# get training and validation steps in one epoch
train_batch = int(np.floor(train.num_samples/batch_size))
valid_bacth = int(np.floor(valid.num_samples/batch_size))

# Start the model
with tf.Session() as sess:
    sess.run(init)
    
    # write the graph to the tensorboard
    writer.add_graph(sess.graph)
    # load the fine-tune weights to model
    model.load_initial_weights(sess)
    
    print("{} start training the model ... ".format(datetime.now()))
    print(" open tensorboard at -- logdir{} .".format(filewriter_path))
    
    for epoch in range(num_epochs):
        print("{} Epoch number: {} ... ".format(datetime.now(), epoch))
        
        # Initialize the iterator for dataset
        sess.run(train_init_op)
        
        for step in range(train_batch):
            img, label = sess.run(next_batch)
            sess.run(train_op, feed_dict = {x: img, y: label, keep_pro = dropout_rate})
            
            if step % display_step == 0:
                s = sess.run(merged_summary, feed_dict = {x: img, y: label, keep_pro = 1.0})
                writer.add_summary(s, epoch*train_batch+step)
            
        # validate the model on the validation dataset
        print("{} start validating the model ...")
        sess.run(valid_inti_op)
        valid_acc = 0
        valid_count = 0
        for _ in range(valid_batch):
            img, label = sess.run(next_batch)
            acc = sess.run(accuracy, feed_dict = {x: img, y: label, keep_pro = 1.0})
            # get accuracy for all data
            valid_acc += acc
            valid_count += 1
            # get the average accuracy
            valid_acc /= valid_count
        print("validation accuracy = {:.4f} ... ".format(valid_acc))
        print("{} saving checkpoints of model ... ".format(datetime.now()))
        # saving checkpoints of model 
        checkpoint_name = os.path.join(checkpoint_pacth, "epcoh " + str(epoch+1) + ".ckpt")
        save_path = saver.save(sess, checkpoint_name)
        print("Model checkpoint saved at {}".format(checkpoint_path))